In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
fileroecsv = r'C:\Users\lzw\Desktop\undergraduate_thesis\rcode\rcode_undergraduate\prelimary_code\data_with_id.csv'
df = pd.read_csv(fileroecsv)

In [4]:
stkcd = 'Stkcd'
Y = 'ViolationTypeID'
var_nam = ['F010101A','F010201A','F011201A','F012301B','F040401B','F040505C','F041101B','F041705C','F050204C','F053201B','F053301C','F080102A','F080603A','F081103B','F082601B','TopTenHoldersRate']
var_nam.insert(0,Y)
var_nam.insert(0,stkcd)
var_dic = {'偿债能力':['F010101A','F010201A','F011201A','F012301B'],
'经营能力':['F040401B','F040505C','F041101B','F041705C'],
'盈利能力':['F050204C','F053201B','F053301C'],
'发展能力':['F080102A','F080603A','F081103B','F082601B'],
'治理结构':['TopTenHoldersRate']}

In [5]:
pro_data = df[var_nam]

In [6]:
#得到z-score
z_path = r'C:\Users\lzw\Desktop\undergraduate_thesis\z_score_2018.xlsx'
z_score = pd.read_excel(z_path)

In [7]:
z_score = z_score.fillna(np.mean(z_score.iloc[:,2]))

In [23]:
pro_data

,Stkcd,ViolationTypeID,F010101A,F010201A,F011201A,F012301B,F040401B,F040505C,F041101B,F041705C,F050204C,F053201B,F053301C,F080102A,F080603A,F081103B,F082601B,TopTenHoldersRate
0,2,3,1.154342,0.485571,0.845856,0.029598,2.520506,0.276039,4.350560,0.221000,0.036580,0.202741,0.374816,1.262205,0.311695,0.319078,0.190692,57.40
1,4,2,1.805236,1.805236,0.477966,-0.422087,0.000000,8.960810,0.822000,1.183406,-0.069716,-0.144284,0.818792,0.995842,0.306249,-3.686165,-0.105459,53.90
2,5,3,1.144871,0.832409,0.494040,-0.062164,0.985190,0.763580,3.609766,0.161160,0.051959,0.132320,0.297082,1.124574,0.071881,1.974936,0.110187,32.04
3,6,3,1.917972,0.733976,0.528236,0.220348,2.562671,0.229199,4.151306,0.188318,0.069209,0.155587,0.387866,1.118930,0.030198,0.113265,0.115237,40.86
4,8,3,2.032443,1.782461,0.292510,-0.180250,0.270103,1.881115,2.196033,0.242605,0.032338,0.065947,0.489184,1.030304,-0.019849,-0.573090,0.044294,40.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3203,900929,3,1.463881,1.463365,0.403461,-0.139804,0.000166,5487.711212,0.471347,0.995981,0.043548,0.130575,0.093444,0.753248,-0.096416,0.928894,0.041821,55.23
3204,900939,3,2.748386,2.748041,0.231591,0.382066,0.000568,12.664887,4.523822,0.134832,0.013755,0.029398,0.678022,1.018791,0.028523,-0.445160,0.017970,54.21
3205,900948,3,1.243217,1.172493,0.549955,0.196488,0.034223,19.017656,0.601589,0.437543,0.057757,0.105402,0.303943,1.135029,0.118150,-0.109987,0.090868,73.86
3206,900956,3,1.136096,0.954085,0.681102,0.065392,0.123782,6.787582,0.772636,0.893800,0.029020,0.094796,0.145279,1.098012,0.033946,0.247022,0.080075,52.67


In [8]:
a = z_score['证券代码'].str.split('.')
Stkcd = []
for stkcd in a:
    Stkcd.append(stkcd[0])
z_score['证券代码'] = Stkcd
z_score['证券代码'] = z_score['证券代码'].astype('int64')

In [34]:
df.dtypes

Stkcd                int64
ViolationTypeID      int64
Ownership            int64
Board                int64
IndustryCodeB       object
                    ...   
F012301B           float64
F012401B           float64
F012501B           float64
F012601B           float64
F012701B           float64
Length: 177, dtype: object

In [9]:
z_score.columns=['Stkcd','name','z_2018']

In [120]:
pro_data = pro_data.merge(z_score[['Stkcd','z_2018']],on='Stkcd',how='left')

In [11]:
bd_2017 = r'C:\Users\lzw\Desktop\bad_debt2017.xlsx'
bd_2018 = r'C:\Users\lzw\Desktop\bad_debt2018.xlsx'
bd_2017 = pd.read_excel(bd_2017)
bd_2018 = pd.read_excel(bd_2018)

In [15]:
bd_2017['账龄'] = bd_2017['账龄'].astype('str')
bool = bd_2017['账龄'].str.contains('期')
a = list(bd_2017[bool]['账龄'].unique())
dict1 = []
for i in a:
    if not re.match(r'[+-]?\d+$',i):
        if '-' not in i:
            dict1.append(i.split('年')[0])
dict2 = [0.08,0,0.5,1.5,1,0,0,0.5,1.5,1,0.5,1,0,1,5,0.5,1]
v = dict(zip(dict1,dict2))

bd_2018['账龄'] = bd_2018['账龄'].astype('str')
bool = bd_2018['账龄'].str.contains('期')
a = list(bd_2018[bool]['账龄'].unique())
dict1 = []
for i in a:
    if '-' not in i:
        # print(i)
        dict1.append(i.split('年')[0])
dict2 = [0.5,0,0.08,1,0,0,0,0.08,0.25,0.5,1,0,0.5,0,0,0,0,0,0,0,0,0,0,0,0.5,1,0,0,1,5,0.5,0.08]
v2 = dict(zip(dict1,dict2))

def get_year(series,v):
    year = []
    for item in series.str.split('年'):
        if item[0] in list(v.keys()):
            year.append(v[item[0]])
            continue
            #返回为True说明不含有数字
        #下面讨论有'-'
        else:
            #没有期
            if '期' not in item[0]:
                if '满5' in item[0]:
                    year.append(5)
                    continue
                if '半' in item[0]:
                    year.append(0.5)
                    continue
                if '--' in item[0]:
                    medi = item[0].split('--')[1]
                    year.append(float(medi))
                    continue
                # print(item)
                year.append(np.mean(np.array(item[0].split('-')).astype('float')))
                continue
            #有期
            if '期' in item[0]:
                if '内' in item[0]:
                    year.append(0)
                    continue
                if '未' in item[0]:
                    year.append(0)
                    continue
                if '信用期-1' in item[0]:
                    year.append(1)
                    continue

                else:
                    medi = item[0].split('期')[1].split('-')[1]
                    year.append(float(medi))
                    continue
    return year

In [17]:
year2017 = get_year(bd_2017['账龄'],v)
len(year2017)
year2018 = get_year(bd_2018['账龄'],v2)
len(year2018)

18047

In [30]:
def covert_stkcd(df):
    z_score = df
    a = z_score['Stkcd'].str.split('.')
    Stkcd = []
    for stkcd in a:
        Stkcd.append(stkcd[0])
    z_score['Stkcd'] = Stkcd
    z_score['Stkcd'] = z_score['Stkcd'].astype('int64')
    return z_score

In [47]:
bd2018 = bd_2018[['证券代码','应收账款提取比例(%)']]
bd2018.columns=['Stkcd','ratio']
bd2018['year2018'] = year2018
bd2018.drop([20812,20813],inplace=True)
bd2018 = covert_stkcd(bd2018)

D:\python\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\python\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
D:\python\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [40]:
bd2017 = bd_2017[['证券代码','应收账款提取比例(%)']]
bd2017.columns=['Stkcd','ratio']
bd2017['year2017'] = year2017
bd2017.drop([18045,18046],inplace=True)
bd2017 = covert_stkcd(bd2017)

D:\python\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [63]:
df = bd2017

In [79]:
Stkcd = list(df.groupby('Stkcd').sum().index)
sumyear = list(df.groupby('Stkcd').sum()['year2017'])
denominator = dict(zip(Stkcd,sumyear))

In [90]:
proportions = []
for index,row in df.iterrows():
    y2017 = row['year2017']
    st = row['Stkcd']
    den = denominator[st]
    proportion = y2017/den
    proportions.append(proportion)

In [92]:
df['proportions']=proportions

D:\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [94]:
def wavg(group,avg_name,weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return(d*w).sum()/w.sum()
    except ZeroDivisionError:
        return d.mean()

df.groupby('Stkcd').apply(wavg,'ratio','year2017')

Stkcd
2         16.833333
4         32.937500
5          5.000000
6         49.642857
7         27.500000
            ...    
900952    44.687500
900953    26.912500
900955    55.125000
900956    55.560000
900957    83.361111
Length: 3361, dtype: float64

In [105]:
b = pd.DataFrame(bd2018.groupby('Stkcd').apply(wavg,'ratio','year2018'))
b.columns = ['bad_debt18']

In [101]:
a = pd.DataFrame(df.groupby('Stkcd').apply(wavg,'ratio','year2017'))
a.columns = ['bad_debt17']

In [110]:
bd = a.merge(b,left_index = True,right_index = True,how='left')

In [112]:
bd['bd_ratio'] = (bd['bad_debt18']-bd['bad_debt17'])/bd['bad_debt17']

In [117]:
bd = bd.fillna(0)

In [118]:
bd.to_csv(r'C:\Users\lzw\Desktop\undergraduate_thesis\bd.csv')

In [140]:
bd = bd[['bad_debt18','bd_ratio']]
pro_data = pro_data.merge(bd,left_on='Stkcd',right_index=True,how='left')

In [122]:
bd

,bad_debt17,bad_debt18,bd_ratio
Stkcd,,,
2,16.833333,0.000000,0.0000
4,32.937500,32.937500,0.0000
5,5.000000,5.000000,0.0000
6,49.642857,49.642857,0.0000
7,27.500000,27.500000,0.0000
...,...,...,...
900952,44.687500,44.687500,0.0000
900953,26.912500,26.912500,0.0000
900955,55.125000,55.125000,0.0000


In [142]:
#补一下缺失值
pro_data = pro_data.fillna(np.mean(pro_data,axis=0))

In [141]:
pro_data

,Stkcd,ViolationTypeID,F010101A,F010201A,F011201A,F012301B,F040401B,F040505C,F041101B,F041705C,...,F080102A,F080603A,F081103B,F082601B,TopTenHoldersRate,z_2018,bad_debt18_x,bd_ratio_x,bad_debt18_y,bd_ratio_y
0,2,3,1.154342,0.485571,0.845856,0.029598,2.520506,0.276039,4.350560,0.221000,...,1.262205,0.311695,0.319078,0.190692,57.40,0.6669,0.000000,0.00000,0.000000,0.00000
1,4,2,1.805236,1.805236,0.477966,-0.422087,0.000000,8.960810,0.822000,1.183406,...,0.995842,0.306249,-3.686165,-0.105459,53.90,6.1534,32.937500,0.00000,32.937500,0.00000
2,5,3,1.144871,0.832409,0.494040,-0.062164,0.985190,0.763580,3.609766,0.161160,...,1.124574,0.071881,1.974936,0.110187,32.04,1.4972,5.000000,0.00000,5.000000,0.00000
3,6,3,1.917972,0.733976,0.528236,0.220348,2.562671,0.229199,4.151306,0.188318,...,1.118930,0.030198,0.113265,0.115237,40.86,1.9481,49.642857,0.00000,49.642857,0.00000
4,8,3,2.032443,1.782461,0.292510,-0.180250,0.270103,1.881115,2.196033,0.242605,...,1.030304,-0.019849,-0.573090,0.044294,40.42,3.1064,61.724986,-0.01677,61.724986,-0.01677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3203,900929,3,1.463881,1.463365,0.403461,-0.139804,0.000166,5487.711212,0.471347,0.995981,...,0.753248,-0.096416,0.928894,0.041821,55.23,3.6079,0.000000,0.00000,NaN,NaN
3204,900939,3,2.748386,2.748041,0.231591,0.382066,0.000568,12.664887,4.523822,0.134832,...,1.018791,0.028523,-0.445160,0.017970,54.21,23.7455,57.812500,0.00000,57.812500,0.00000
3205,900948,3,1.243217,1.172493,0.549955,0.196488,0.034223,19.017656,0.601589,0.437543,...,1.135029,0.118150,-0.109987,0.090868,73.86,1.4230,0.000000,0.00000,NaN,NaN
3206,900956,3,1.136096,0.954085,0.681102,0.065392,0.123782,6.787582,0.772636,0.893800,...,1.098012,0.033946,0.247022,0.080075,52.67,1.6780,55.560000,0.00000,55.560000,0.00000


In [144]:
pro_data.to_csv(r'C:\Users\lzw\Desktop\undergraduate_thesis\data_with_id.csv')

In [51]:
bd2018['year2018'].unique().max()
bd2018['year2018'].unique().min()

0.0